# Task 1

## Demographic parity

Probability of being enrolled in training for a student from blue population is 0.65 and from the red one is 0.5. Dividing the two numbers, we get the ratio: 130%.

## Equal opportunity

Probability of being enrolled for a student from the blue group that will find XAI useful is 0.75. The same calculation for a student from red group is 0.5. We get the ratio of 150%.

## Predictive rate parity

### Positive

Probability of finding XAI useful in the future if we know that student from the blue group was enrolled is 60/65. The same number for red group is 1/2. We get the ratio of approximately 185%.

### Negative

Probability of finding XAI useful in the future if we know that student from the blue group was not enrolled is 20/35. The same number for red group is 1/2. We get the ratio of approximately 114%.

# Task 2

In this part I will be working with the Give Me Some Credit dataset. I will check fairness regarding to Age attribute, as this characteristic is probably important in our case. I will analyze two groups: below 60 years old (privileged) and more than 60 years old.

## Part 2

First checked model is Random Forest Classifier with default hyperparameters. The model has accuracy greater than 90% and therefore can be though of as a good predictor in our case. 
We can see that Statistical Parity is 0.322581. Predictive parity is 1.087954. Equal opportunity is 0.868852. Our model is biased with regard to Statistical Parity.
The results are shown on the plot below.

In [23]:
part_2()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'young'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV   FPR       STP
old  0.868852  1.057987  1.087954  0.25  0.322581


![Alt text](part_2.png)

## Part 3

Second checked model is Logistic Regression Classifier with default hyperparameters. Again, the model has accuracy greater than 90% and therefore can be though of as a good predictor in our case. 
We can see that Statistical Parity is 0.111111. Predictive parity is 0.666. Equal opportunity is 0.098039. Our model is biased with regard to Statistical Parity, Predictive Parity and Equal Opportunity.

The results are shown on the plot below. In on of the metrics (FPR) we got a NaN value, therefore we can take a look also at the Metric Scores plot. It turns out that the model is not biased according to this metric.

In [31]:
part_3()

Bias detected in 3 metrics: TPR, PPV, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'young'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC    PPV  FPR       STP
old  0.098039  1.056955  0.666  NaN  0.111111

Warning!
Take into consideration that NaN's are present, consider checking 'metric_scores' plot to see the difference


![Alt text](part_3.png)

![Alt text](part_3_2.png)

## Part 4

I have chosen to use the roc-pivot bias mitigation technique. Comparison of results for model with and without bias mitigation is shown below. We can see that bias is signifficantly reduced in case of Predictive Equality and Statistical Parity. Of course, it's usually not possible to balance the model in all possible directions. We can see that Equal Opportunity gets worse in this case.

In [25]:
part_4()

Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'young'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC   PPV       FPR       STP
old  1.326797  1.055738  0.92  0.583333  0.583333


![Alt text](part_4.png)

## Part 5

In this part I compare performance of models and it's correlation with fairness. We can observe that accuracy of both classifiers without bias mitigation is almost indistinguishable. Interestingly, the forest model with bias mitigation technique applied (and therefore with increased fairness) achieves slightly better performance. This suggests that we have - in a sense - got a better model, that is more fair with respect to the analyzed attricute and at the same time more accurate.

In [35]:
part_5()

--------- Accuracy of forest before bias mitigation: ---------
                          recall  precision        f1  accuracy       auc
RandomForestClassifier  0.178129   0.526423  0.266187  0.929548  0.833767



--------- Accuracy of forest after bias mitigation: ---------
                          recall  precision        f1  accuracy       auc
RandomForestClassifier  0.154746   0.552826  0.241805  0.930386  0.833825



--------- Accuracy of Logistic Regression: ---------
                      recall  precision        f1  accuracy       auc
LogisticRegression  0.044017   0.496124  0.080859  0.928216  0.687267


# Appendix

In [34]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from dalex import Explainer
from dalex.fairness import roc_pivot
import numpy as np

df = pd.read_csv('cs-training.csv')
df = df.iloc[:, 1:]

df = df.dropna()
X = df.iloc[:100000, 1:]
y = df.iloc[:100000, 0]
X_test = df.iloc[100000:, 1:]
y_test = df.iloc[100000:, 0]
model = RandomForestClassifier().fit(X, y)
protected = np.where(X_test.age < 60, 'young', 'old')
privileged = 'young'

def predict_fn(model, X):
    return model.predict_proba(X)[:, 1]

forest_exp = Explainer(model, X_test, y_test, predict_function=predict_fn, verbose=False)
forest_fobject = forest_exp.model_fairness(protected = protected, privileged=privileged, label='forest_base')

def part_2():
    forest_fobject.fairness_check()
    forest_fobject.plot()

lr_model = LogisticRegression(max_iter=1000).fit(X, y)

exp = Explainer(lr_model, X_test, y_test, predict_function=predict_fn, verbose=False)
exp.model_performance().result

def part_3():    
    fobject = exp.model_fairness(protected = protected, privileged=privileged)
    fobject.fairness_check()
    fobject.plot()
    fobject.plot(type='metric_scores')

roc_exp = roc_pivot(forest_exp, protected=protected, privileged=privileged)
roc_fobject = roc_exp.model_fairness(protected=protected, privileged=privileged, label='forest_roc')

def part_4():
    roc_fobject.fairness_check()
    roc_fobject.plot([forest_fobject])

def part_5():
    print('--------- Accuracy of forest before bias mitigation: ---------')
    print(forest_exp.model_performance().result)
    print('\n\n\n--------- Accuracy of forest after bias mitigation: ---------')
    print(roc_exp.model_performance().result)
    print('\n\n\n--------- Accuracy of Logistic Regression: ---------')
    print(exp.model_performance().result)

/home/kuba/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



protected array is not string type, converting to string 


/home/kuba/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names

